In [ ]:
import sys
sys.path.insert(0, '/Users/rdchlth9/Codes/cmtb_refactor/')
from getdatatestbed import getDataFRF
from prepdata.prepDataLib import PrepDataTools as preptools
from prepdata import inputOutput
from prepdata import writeRunRead as wrr
from testbedutils import fileHandling

import datetime
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc4
import os
import plotly
import pickle
import yaml
%matplotlib inline

In [ ]:
d1 = datetime.datetime(2018,3,5,0,0,0)
d2 = datetime.datetime(2018,3,6,0,0,0)
simduration = 24.
ftime = 24*3600.
dt = 3600. #seconds has to be a float and not an integer
reltime = np.arange(0,ftime+dt,dt)
fname = 'march_test'

In [ ]:
go = getDataFRF.getObs(d1-datetime.timedelta(hours=3),d2+datetime.timedelta(hours=3))

In [ ]:
#rawWL = go.getWL()
#rawwave_data8m = go.getWaveSpec('8m-array')
rawwave_data6m = go.getWaveData('awac-6m',spec=True)
#rawwind = go.getWind()

In [ ]:
#cmtb_data = getDataFRF.getDataTestBed(d1, d1+datetime.timedelta(minutes=1))
#bathy_data = cmtb_data.getBathyIntegratedTransect()

In [ ]:
prepdata = preptools()

this is just a simple function to alleviate some clutter in frontBackCSHORE.  It basically takes in the
        wave gauge data from the 8m and 6m gauges, as well as the prev wave gauge if one is available, then decides
        which data need to be used

In [ ]:
#creates processed wave packet for cshore
waves = prepdata.prep_CSHOREwaves(rawwave_data6m, reltime , d1)

In [ ]:
#Bathy stuff is really slow right now. look into it

In [ ]:
bathy_loc = 'survey'
profile_num = 960
dx = 1
fric_fac = 0.015

In [ ]:
#prof_nums = go.getBathyTransectProfNum()
#print(prof_nums)

In [ ]:
bathy_data = go.getBathyTransectFromNC(profilenumbers=960)

In [ ]:
with open('bathysave.p', 'wb') as fid:
    pickle.dump(bathy_data, fid, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
b_dict = prepdata.prep_CSHOREbathy(bathy_data, bathy_loc, profile_num, dx, waves, fric_fac)
print(bathy_data)
print(b_dict)

## get water levels

In [ ]:
rawWL = go.getWL()

In [ ]:
wlTimeList = [d1 + datetime.timedelta(seconds=tt) for tt in reltime]
wl_data = prepdata.prep_WL(rawWL,wlTimeList)

In [ ]:
ctd_data = go.getCTD()

In [ ]:
wl_data

## make input files

In [ ]:
cshoreio = wrr.cshoreio(pathPrefix='/Users/rdchlth9/Codes/cmtb_refactor/notetbooks/cshore/data', fNameBase=fname+'_cmtb', versionPrefix='base',
                     startTime=d1, endTime=d2, runFlag=True,
                     generateFlag=True, readFlag=False)

In [ ]:
if not os.path.isdir(cshoreio.workingDirectory):
    os.makedirs(cshoreio.workingDirectory)

In [ ]:
cshoreio.setCSHORE_time(dt)

In [ ]:
cshoreio.readCSHORE_nml(nmlname='namelists.nml')

In [ ]:
cshoreio.makeCSHORE_meta(b_dict,waves,wl_data)

In [ ]:
cshoreio.makeCSHORE_bc(b_dict,waves,wlDict=wl_data)

In [ ]:
cshoreio.writeCSHORE_infile()

## run cshore

In [ ]:
cshoreio.runSimulation('/Users/rdchlth9/Public/cshore/src-repo/cshore.exe')

## read cshore results

In [ ]:
# path needs FIXED or MOBILE in the name, I put in bandaid but this needs to be fixed on line 4978 too
cshoreio.readCSHORE_ODOC()
cshoreio.readCSHORE_infile()
cshoreio.readCSHORE_OBPROF()
cshoreio.readCSHORE_OSETUP()
cshoreio.readCSHORE_OXVELO()
cshoreio.readCSHORE_OYVELO()
cshoreio.readCSHORE_OCROSS()
cshoreio.readCSHORE_OLONGS()
cshoreio.readCSHORE_OBSUSL()

In [ ]:
cshoreio.gatherCSHORE_params()
cshoreio.gatherCSHORE_bc()
cshoreio.gatherCSHORE_hydro()
cshoreio.gatherCSHORE_sed()
cshoreio.gatherCSHORE_veg()
cshoreio.gatherCSHORE_morpho()
cshoreio.gatherCSHORE_meta()

In [ ]:
results = cshoreio._resultsToDict()

In [ ]:
## plot runup

In [ ]:
cshoreiofull = wrr.cshoreio(pathPrefix='/Users/rdchlth9/Codes/cmtb_refactor/notetbooks/cshore/data', fNameBase=fname+'_cmtb_full', versionPrefix='base',
                     startTime=d1, endTime=d2, timeStep=dt, runFlag=True,
                     generateFlag=True, readFlag=False)

In [ ]:
cshoreiofull.infile

In [ ]:
fileHandling.makeCMTBfileStructure(cshoreiofull.workingDirectory)

In [ ]:
cshoreiofull.writeAllFiles(b_dict,waves,wlPacket=wl_data,ctdPacket=ctd_data,newModelParams={'gamma':0.64})

In [ ]:
cshoreiofull.infile

In [ ]:
cshoreiofull.runSimulation('/Users/rdchlth9/Public/cshore/src-repo/cshore.exe')

In [ ]:
resultsfull = cshoreiofull.readAllFiles()

In [ ]:
resultsfull['params']

In [ ]:
yy = {}
yy['THREDDS'] = 'CHL'
yy['bathyLoc'] = 'integrated_bathy'
yy['startTime'] = datetime.datetime.strftime(d1,'%Y-%m-%dT%H:%M:%SZ')
yy['endTime'] = datetime.datetime.strftime(d2,'%Y-%m-%dT%H:%M:%SZ')
yy['modelSettings'] = {}
yy['modelSettings']['version_prefix'] = 'base'
yy['modelSettings']['modelName'] = 'cshore'
yy['profileNumber'] = profile_num
yy['simulationDuration'] = simduration
yy['modelExecutable'] = '/Users/rdchlth9/Public/cshore/src-repo/cshore.exe'
yy['workingDirectory'] = '/data'
yy['netCDFdir'] = '/thredds_data'
yy['logfileLoc'] = '/cmtb_refactor/data'
yy['plotFlag'] = False
yy['generateFlag'] = True
yy['runFlag'] = True
yy['analyzeFlag'] = True

In [ ]:
stream = open('../../test.yaml', 'w')
yaml.dump(yy,stream)

In [ ]:
%run ../../genericWorkFlow.py test.yaml